In [8]:
# Import the library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder



In [9]:
# LOAD the dataset

# Read the CSV file into a DataFrame
df = pd.read_csv('LengthOfStay.csv')

# Display the first few rows of the DataFrame
print(df.head())

   eid       vdate rcount gender  dialysisrenalendstage  asthma  irondef  \
0    1   8/29/2012      0      F                      0       0        0   
1    2   5/26/2012     5+      F                      0       0        0   
2    3   9/22/2012      1      F                      0       0        0   
3    4    8/9/2012      0      F                      0       0        0   
4    5  12/20/2012      0      F                      0       0        0   

   pneum  substancedependence  psychologicaldisordermajor  ...     glucose  \
0      0                    0                           0  ...  192.476918   
1      0                    0                           0  ...   94.078507   
2      0                    0                           0  ...  130.530524   
3      0                    0                           0  ...  163.377028   
4      1                    0                           1  ...   94.886654   

   bloodureanitro  creatinine        bmi  pulse  respiration  \
0         

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   eid                         100000 non-null  int64  
 1   vdate                       100000 non-null  object 
 2   rcount                      100000 non-null  object 
 3   gender                      100000 non-null  object 
 4   dialysisrenalendstage       100000 non-null  int64  
 5   asthma                      100000 non-null  int64  
 6   irondef                     100000 non-null  int64  
 7   pneum                       100000 non-null  int64  
 8   substancedependence         100000 non-null  int64  
 9   psychologicaldisordermajor  100000 non-null  int64  
 10  depress                     100000 non-null  int64  
 11  psychother                  100000 non-null  int64  
 12  fibrosisandother            100000 non-null  int64  
 13  malnutrition   

In [11]:
# 2. Classify Length of Stay into "Short", "Medium", and "Long" using qcut
df['lengthofstay_class'] = pd.qcut(df['lengthofstay'], q=3, labels=['Short', 'Medium', 'Long'])

In [12]:
# 3. Encode target labels into numeric format (0, 1, 2)
le_target = LabelEncoder()
df['lengthofstay_class'] = le_target.fit_transform(df['lengthofstay_class'])

In [13]:
# Convert 'vdate' and 'discharged' to datetime
df['vdate'] = pd.to_datetime(df['vdate'])
df['discharged'] = pd.to_datetime(df['discharged'])

In [14]:
# Handle the '5+' value in the 'rcount' column
df['rcount'] = df['rcount'].replace('5+', 5)  # You can replace with a different value if necessary
df['rcount'] = pd.to_numeric(df['rcount'], errors='coerce')  # Convert to numeric, invalid parsing will be set to NaN

# Check for any remaining missing values after conversion
print(df['rcount'].isnull().sum())

0


In [15]:
# 4. Encoding Categorical Features
from sklearn.preprocessing import StandardScaler, LabelEncoder
binary_columns = ['gender', 'dialysisrenalendstage', 'asthma', 'irondef', 'pneum', 
                  'substancedependence', 'psychologicaldisordermajor', 'depress', 
                  'psychother', 'fibrosisandother', 'malnutrition', 'hemo', 
                  'secondarydiagnosisnonicd9']

le = LabelEncoder()

for col in binary_columns:
    df[col] = le.fit_transform(df[col])

In [16]:
# 5. Scaling Numerical Features
scaler = StandardScaler()
numerical_features = ['hematocrit', 'neutrophils', 'sodium', 'glucose', 'bloodureanitro',
                      'creatinine', 'bmi', 'pulse', 'respiration']

df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [17]:
#6. Split Data into Training and Test Sets
from sklearn.model_selection import train_test_split
X = df.drop(['lengthofstay', 'eid', 'vdate', 'discharged', 'facid', 'lengthofstay_class'], axis=1)  # Features
y = df['lengthofstay_class']  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# 7. Model Training - Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 8. Model Evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Model Accuracy: {accuracy * 100:.2f}%')


# Get feature importance
feature_importance = rf.feature_importances_

# Create a DataFrame for better visualization
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False)

# Display feature importance
print(feature_importance_df)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.show()

Model Accuracy: 88.97%


NameError: name 'rf' is not defined

In [ ]:
# 7. Train the XGBoost Model
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
model = xgb.XGBClassifier(objective='multi:softmax', num_class=3, random_state=42)
model.fit(X_train, y_train)

# 8. Model Evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'XGBoost Model Accuracy: {accuracy * 100:.2f}%')
print("Classification Report:\n", classification_report(y_test, y_pred))

# 9. Hyperparameter Tuning using GridSearchCV
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', 
                           cv=StratifiedKFold(n_splits=5), verbose=1, n_jobs=-1)

grid_search.fit(X_train, y_train)

# Best hyperparameters
print(f"Best Hyperparameters: {grid_search.best_params_}")

# Train model with the best hyperparameters
best_model = grid_search.best_estimator_

# Evaluate the tuned model
y_pred_tuned = best_model.predict(X_test)
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)

print(f'XGBoost Model (Tuned) Accuracy: {accuracy_tuned * 100:.2f}%')
print("Classification Report (Tuned):\n", classification_report(y_test, y_pred_tuned))
